In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "DEV")
dbutils.widgets.text("target_dataset", "VCM_DWH_PRD")
dbutils.widgets.text("catalog_name", "udp_wcm_dev")
dbutils.widgets.text("metadata_schema", "udp_wcm_metadata_dev")
dbutils.widgets.text("target_table", "f_sale_prod_online_excl_dichoho")
dbutils.widgets.text(
    "dependency_table","VCM_DWH_PRD.F_SALE_PROD",
)

In [0]:
dbutils.widgets.text("field_calday", "CALDAY")
dbutils.widgets.text("field_id", "HASH_ID")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
environment = dbutils.widgets.get("environment")
target_dataset = dbutils.widgets.get("target_dataset")
target_table = dbutils.widgets.get("target_table")
metadata_schema = dbutils.widgets.get("metadata_schema")
dependency_table = dbutils.widgets.get("dependency_table")
dependency_table = [x.strip().upper() for x in dependency_table.split(",")]
dependency_table = "'" + "','".join(dependency_table) + "'"

print(f"environment: {environment}")
print(f"target_dataset: {target_dataset}")
print(f"target_table: {target_table}")
print(f"catalog_name: {catalog_name}")
print(f"metadata_schema: {metadata_schema}")
print(f"dependency_table: {dependency_table}")

In [0]:
field_calday = dbutils.widgets.get("field_calday")
field_id = dbutils.widgets.get("field_id")

print(f"field_calday: {field_calday}")
print(f"field_id: {field_id}")

In [0]:
%run "../common/common_etl_load"

In [0]:
%python
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_gold_vcm_dwh.f_sale_prod_online_excl_dichoho
(
  hash_id BIGINT,
  calday DATE,
  base_uom STRING,
  sales_unit STRING,
  transaction_index STRING,
  transaction_id STRING,
  product_id STRING,
  business_unit STRING,
  store_id STRING,
  promotion STRING,
  transaction_no STRING,
  transaction_type STRING,
  time_of_day INT,
  sale_time INT,
  credit_card_id STRING,
  payment_method STRING,
  department STRING,
  pos_number STRING,
  offer STRING,
  discount_type STRING,
  discount_type_group STRING,
  recordmode STRING,
  return_qty1 DOUBLE,
  return_qty2 DOUBLE,
  return_value DOUBLE,
  promotion_qty DOUBLE,
  base_sale_qty DOUBLE,
  receipt_count DOUBLE,
  sale_in_cost_value DOUBLE,
  promotion_sale DOUBLE,
  sale_qty DOUBLE,
  sale_amount DOUBLE,
  promotion_sale_l DOUBLE,
  return_sale_l DOUBLE,
  discount_item_no DOUBLE,
  discount_value DOUBLE,
  tax_amount DOUBLE,
  tender_value DOUBLE,
  vinid_card_no STRING,
  vat_of_redeem DOUBLE
)
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
""")

In [0]:
if CALDAY_IN_STR != '':
    spark.sql(f"""
    CREATE OR REPLACE TEMPORARY VIEW temp_{target_table} AS
    SELECT
        a.* EXCEPT (IS_SLL),
        IF(a.DISCOUNT_TYPE = 'ZRTA', a.DISCOUNT_VALUE, IF(a.DISCOUNT_TYPE = 'ZCRT', a.DISCOUNT_VALUE, 0)) VAT_OF_REDEEM
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_sale_prod a
    WHERE
        REGEXP_REPLACE(pos_number, '^0+', '') IN ('222', '333', '555', '777', '999')
         AND (a.calday IN ({CALDAY_IN_STR}) AND a.CALDAY < DATE(CURRENT_TIMESTAMP() +  INTERVAL 7 HOURS))
    """)
else:
    print(f"No valid calday's found In common-etl to create the view: temp_{target_table}")

In [0]:
# Get the calday list string from temp table
if CALDAY_IN_STR != '':
    calday_query = f"""
    SELECT
        CONCAT("'", CONCAT_WS("','", COLLECT_SET(CAST(TO_DATE(calday, 'yyyy-MM-dd') AS STRING))), "'") AS calday_in_str
    FROM temp_{target_table}
    """

    calday_df = spark.sql(calday_query)
    calday_in_str = calday_df.collect()[0][0]

    # Only run DELETE if the string is non-empty
    if calday_in_str.strip("'"):  # checks if there's any date inside the quotes
        spark.sql(f"""
        DELETE FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_sale_prod_online_excl_dichoho
        WHERE calday IN ({calday_in_str})
        """)
    else:
        print("No valid CALDAYs found to delete.")

In [0]:
if CALDAY_IN_STR != "":
    # Insert rows from temp table into main table
    INSERT_ROW = ", ".join(
        spark.table(f"{catalog_name}.udp_wcm_gold_vcm_dwh.f_sale_prod_online_excl_dichoho").columns
    )
    spark.sql(
        f"""
        INSERT INTO {catalog_name}.udp_wcm_gold_vcm_dwh.f_sale_prod_online_excl_dichoho ({INSERT_ROW})
        SELECT {INSERT_ROW} FROM temp_{target_table}
    """
    ).display()
else:
    print(
        f"temp_{target_table} could't be created as no valid calday's found In common-etl so {catalog_name}.udp_wcm_gold_vcm_dwh.f_sale_prod_online_excl_dichoho can't be inserted using calday's from temp_{target_table}"
    )

In [0]:
%run "../common/common_etl_update"

In [0]:
spark.sql(f"""DROP VIEW IF EXISTS temp_{target_table}""")